In [1]:
import numpy as np
import pandas as pd
import os  # for general OS functions if needed
import re




##########
# This file is used to clean and consolidate daily parking transaction excel sheets.
# As of 02/16/2025, the excel files passed must contain multiple sheets within to properly consolidate.
# Use consolidate_Folder(file_path) to combine all transactions in a local folder.
##########




#@title FindHeader, FindFooter, FindStation Rows code


# Check where the first row of transactions begin
def findHeader(df):
    # Search for the header row of the sheet and return the row number
    row = df.index[df.iloc[:, 0] == "Entry Time"]
    return row[0] + 1  # Returns the row number of the header row, you should skip to row[0]-1


# Check where the last row of transactions begin
def findFooter(df):
    # Search for the footer row of the sheet and return the number of footer rows to skip
    row = df.index[df.iloc[:, 0].str.contains("Totals", case=False, na=False)]
    totalrows = len(df)
    numFooterRows = totalrows - row[0]
    return numFooterRows


# Check to see if a sheet has a station and returns row where station is
def findStation(df):
    row = df.index[df.iloc[:, 0].str.contains("Station", case=False, na=False)]
    if row.empty:
        return False
    else:
        return row[0]




# Consolidated Code


# Create a singular process code
def processSheetwithHeader(eFile, sName, oFile, adjust_footer=False):
    print("Processing with header:", eFile)
    df = pd.read_excel(eFile, sheet_name=sName)
    rowHeader = findHeader(df)
    rowFooter = findFooter(df)
    StationRow = findStation(df)


    # Pull out station value for sheet if it exists
    if StationRow != False:
        station = df.iloc[StationRow, 0]
        start_index = station.find("Station: ") + len("Station: ")  # Index of 'A' in 'A21'
        end_index = station.find("-", start_index)  # Index of '-' character
        Station = station[start_index:end_index]


    df = pd.read_excel(eFile, sheet_name=sName, skiprows=rowHeader, skipfooter=rowFooter+1)
    if StationRow != False:  # Create a new column to specify Exit Station
        df.insert(df.columns.get_loc("Prev Stn") + 1, 'Station', Station)
    df.rename(columns={'Trans Time': 'Transaction Time',
                       'Media\n': 'Media',
                       'Media ID\n': 'Media',
                       'Prev Stn': 'Previous Station',
                       'Trans Type': 'Transaction Type'}, inplace=True)
    # You can add or delete columns that you would like to include/exclude in the final df
    df = df[['Entry Time', 'Transaction Time', 'Previous Station', 'Station', 'Media', 'Transaction Type', 'Revenue']]
    df.to_csv(oFile, mode="a", index=False, header=True)  # Write to the correct output file




def processSheetnoHeader(eFile, sName, oFile, adjust_footer=False):
    print("Processing without header:", eFile)
    df = pd.read_excel(eFile, sheet_name=sName)
    rowHeader = findHeader(df)
    rowFooter = findFooter(df)
    if adjust_footer:
        rowFooter = rowFooter + 1
    StationRow = findStation(df)


    # Pull out station value for sheet if it exists
    if StationRow != False:
        station = df.iloc[StationRow, 0]
        start_index = station.find("Station: ") + len("Station: ")  # Index of 'A' in 'A21'
        end_index = station.find("-", start_index)  # Index of '-' character
        Station = station[start_index:end_index]


    df = pd.read_excel(eFile, sheet_name=sName, skiprows=rowHeader, skipfooter=rowFooter)
    if StationRow != False:
        df.insert(df.columns.get_loc("Prev Stn") + 1, 'Station', Station)
    df.rename(columns={'Trans Time': 'Transaction Time',
                       'Media\n': 'Media',
                       'Media ID\n': 'Media',
                       'Prev Stn': 'Previous Station',
                       'Trans Type': 'Transaction Type'},
              inplace=True)
    df = df[['Entry Time', 'Transaction Time', 'Previous Station', 'Station', 'Media', 'Transaction Type', 'Revenue']]
    df.to_csv(oFile, mode="a", index=False, header=False)  # Append without headers




# Consolidates multiple sheets
def consolidateMultipleSheetswithHeadersandFooters(fileList, folder_path, oFile):
    firstFile = fileList[0]


    for eFile in fileList:
        neweFile = os.path.join(folder_path, eFile)  # Combine the folder path with the file name


        print("Processing File:" + str(neweFile))
        dfChecks = pd.ExcelFile(neweFile)
        numSheets = len(dfChecks.sheet_names) + 1


        # Determine if we need to adjust the footer rows
        adjust_footer = True if numSheets == 3 else False


        # If file is the first file, pull out header
        if eFile == firstFile:
            # Check how many sheets are in the eFile
            for i in range(2, numSheets):
                print(i)
                cSheet = "Sheet" + str(i)
                if i == 2:
                    processSheetwithHeader(neweFile, cSheet, oFile, adjust_footer)
                else:
                    processSheetnoHeader(neweFile, cSheet, oFile, adjust_footer)
        else:
            for i in range(2, numSheets):
                print(i)
                cSheet = "Sheet" + str(i)
                processSheetnoHeader(neweFile, cSheet, oFile, adjust_footer)



# Takes a folder as an input and consolidates all excel files
def consolidateFolder(folder_path):
    # Get the list of all files in the folder
    fileList = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
   
    # Define the output file path for the consolidated CSV file
    output_file = os.path.join(folder_path, "consolidated_data.csv")  # Output file remains constant
   
    # Call the consolidate function, passing the list of files and the output file path
    consolidateMultipleSheetswithHeadersandFooters(fileList, folder_path, output_file)  # Pass the folder path and output file directly
   
    return output_file  # Return the path to the consolidated CSV file



# To Run:
consolidateFolder(r'C:\Users\mvalsania25\Desktop\Destination_X')




Processing File:C:\Users\mvalsania25\Desktop\Destination_X\CRITEERIA 03.19.2023.xls
2
Processing with header: C:\Users\mvalsania25\Desktop\Destination_X\CRITEERIA 03.19.2023.xls
Processing File:C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA    08-26-2023.xls
2
Processing without header: C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA    08-26-2023.xls
Processing File:C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA   06.18.2023.xls
2
Processing without header: C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA   06.18.2023.xls
Processing File:C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA   08.30.2023.xls
2
Processing without header: C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA   08.30.2023.xls
Processing File:C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA  11.05.2023.xls
2
Processing without header: C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA  11.05.2023.xls
Processing File:C:\Users\mvalsania25\Desktop\Destination_X\CRITERIA  11.08.2023.xls
2
Proc

'C:\\Users\\mvalsania25\\Desktop\\Destination_X\\consolidated_data.csv'